# Filter out hotspot mutations
Hotspot:
E542K
E535K
H1047R


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
en = cptac.Endometrial()

In [3]:
gene = 'PIK3CA'

In [4]:
help(en.get_genotype_all_vars)

Help on method get_genotype_all_vars in module cptac.dataset:

get_genotype_all_vars(mutations_genes, mutations_filter=None, show_location=True, mutation_hotspot=None) method of cptac.endometrial.Endometrial instance
    Return a dataframe that has the mutation type and wheather or not it is a multiple mutation
    Parameters:
    mutation_genes (str, or list or array-like of str): The gene(s) to get mutation data for.
    mutations_filter (list, optional):  List of mutations to prioritize when filtering out multiple mutations, in order of priority.
    show_location (bool, optional): Whether to include the Location column from the mutation dataframe. Defaults to True.
    mutation_hotspot (optional): a list of hotspots



In [5]:
mut = en.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E535K', 'H1047R'])
prot = en.get_proteomics()
protfilter = prot.index.str.endswith('.N')
prot = prot[~protfilter]


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)


In [6]:
joined = mut.join(prot)
joined

Name,Mutation,Location,Mutation_Status,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,p.E545K,Single_mutation,-1.180,-0.8630,-0.8020,0.2220,0.2560,0.6650,1.2800,...,-0.08770,NaN,0.02290,0.1090,NaN,-0.332,-0.43300,-1.020,-0.12300,-0.0859
C3L-00008,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-0.685,-1.0700,-0.6840,0.9840,0.1350,0.3340,1.3000,...,-0.03560,NaN,0.36300,1.0700,0.7370,-0.564,-0.00461,-1.130,-0.07570,-0.4730
C3L-00032,Missense_Mutation,p.E545K,Single_mutation,-0.528,-1.3200,0.4350,NaN,-0.2400,1.0400,-0.0213,...,0.00112,-0.1450,0.01050,-0.1160,NaN,0.151,-0.07400,-0.540,0.32000,-0.4190
C3L-00090,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-1.670,-1.1900,-0.4430,0.2430,-0.0993,0.7570,0.7400,...,0.07250,-0.0552,-0.07140,0.0933,0.1560,-0.398,-0.07520,-0.797,-0.03010,-0.4670
C3L-00098,Amplification,Amplification,Single_mutation,-0.374,-0.0206,-0.5370,0.3110,0.3750,0.0131,-1.1000,...,-0.17600,NaN,-1.22000,-0.5620,0.9370,-0.646,0.20700,-1.850,-0.17600,0.0513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-01520,Missense_Mutation,p.E726K,Single_mutation,-1.070,-0.7120,0.0462,-0.0471,0.2270,1.3500,1.2100,...,-0.06990,-0.4010,0.55700,0.8270,0.3480,-0.187,-0.10700,-0.830,0.06200,-0.5280
C3N-01521,Amplification,Amplification,Single_mutation,-1.280,-0.7360,-0.5520,0.1140,0.2730,1.0700,0.8500,...,0.13100,NaN,-0.00364,0.7630,0.0784,-0.203,0.01320,-1.200,-0.35700,0.0299
C3N-01537,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-0.290,-0.3200,3.1700,-0.9070,0.0317,-0.0425,NaN,...,-0.17900,-0.5210,0.05230,0.5150,0.5850,0.138,-0.19000,-0.966,-0.00627,-0.2490


In [7]:
joined[joined.Location.str.contains('H1047R')]

Name,Mutation,Location,Mutation_Status,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00362,Missense_Mutation,p.H1047R,Single_mutation,-0.9240,-0.445,1.57,-0.678,0.173,0.436,0.2270,...,-0.0496,NaN,0.1080,-0.153,NaN,0.109,-0.3360,-0.8220,-0.0338,0.12100
C3N-00324,Missense_Mutation,p.H1047R,Single_mutation,-0.4490,-0.440,1.15,NaN,-0.645,0.316,0.0809,...,0.0449,0.0452,-0.1520,-0.197,0.295,-0.188,-0.2650,0.2500,-0.1130,-0.30200
C3N-00383,Missense_Mutation,p.H1047R,Single_mutation,0.0729,0.567,1.71,NaN,-0.278,0.646,0.3780,...,0.2790,-0.3780,0.0125,-0.305,NaN,0.149,-0.0526,-0.0311,0.4320,0.00801
C3N-01212,Missense_Mutation,p.H1047R,Single_mutation,0.0308,-0.142,NaN,NaN,0.211,0.927,0.4920,...,0.0406,NaN,0.0924,0.121,0.433,-1.050,-0.3160,-0.4000,0.1230,-0.39100


In [8]:
joined[joined.Location.str.contains('H1047R')]

Name,Mutation,Location,Mutation_Status,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00362,Missense_Mutation,p.H1047R,Single_mutation,-0.9240,-0.445,1.57,-0.678,0.173,0.436,0.2270,...,-0.0496,NaN,0.1080,-0.153,NaN,0.109,-0.3360,-0.8220,-0.0338,0.12100
C3N-00324,Missense_Mutation,p.H1047R,Single_mutation,-0.4490,-0.440,1.15,NaN,-0.645,0.316,0.0809,...,0.0449,0.0452,-0.1520,-0.197,0.295,-0.188,-0.2650,0.2500,-0.1130,-0.30200
C3N-00383,Missense_Mutation,p.H1047R,Single_mutation,0.0729,0.567,1.71,NaN,-0.278,0.646,0.3780,...,0.2790,-0.3780,0.0125,-0.305,NaN,0.149,-0.0526,-0.0311,0.4320,0.00801
C3N-01212,Missense_Mutation,p.H1047R,Single_mutation,0.0308,-0.142,NaN,NaN,0.211,0.927,0.4920,...,0.0406,NaN,0.0924,0.121,0.433,-1.050,-0.3160,-0.4000,0.1230,-0.39100


# Endo get difference in median and t-test

In [9]:
en.get_proteomics()

Name,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,AAK1,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,-1.1800,-0.8630,-0.802,0.222,0.256000,0.6650,1.2800,-0.3390,0.412,-0.66400,...,-0.08770,NaN,0.0229,0.1090,NaN,-0.33200,-0.43300,-1.020,-0.1230,-0.085900
C3L-00008,-0.6850,-1.0700,-0.684,0.984,0.135000,0.3340,1.3000,0.1390,1.330,-0.36700,...,-0.03560,NaN,0.3630,1.0700,0.737,-0.56400,-0.00461,-1.130,-0.0757,-0.473000
C3L-00032,-0.5280,-1.3200,0.435,NaN,-0.240000,1.0400,-0.0213,-0.0479,0.419,-0.50000,...,0.00112,-0.1450,0.0105,-0.1160,NaN,0.15100,-0.07400,-0.540,0.3200,-0.419000
C3L-00090,-1.6700,-1.1900,-0.443,0.243,-0.099300,0.7570,0.7400,-0.9290,0.229,-0.22300,...,0.07250,-0.0552,-0.0714,0.0933,0.156,-0.39800,-0.07520,-0.797,-0.0301,-0.467000
C3L-00098,-0.3740,-0.0206,-0.537,0.311,0.375000,0.0131,-1.1000,NaN,0.565,-0.10100,...,-0.17600,NaN,-1.2200,-0.5620,0.937,-0.64600,0.20700,-1.850,-0.1760,0.051300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NX5.N,-0.0014,0.6160,-0.641,-0.204,0.027600,-1.4100,-0.4050,-0.5610,-0.373,0.12400,...,0.21500,0.1070,-0.1900,-1.3600,-1.100,-0.19700,-0.30700,0.682,0.0984,-0.020200
NX6.N,0.2980,0.3950,-1.240,-0.178,-0.180000,-0.5910,-0.5980,0.4020,-0.384,0.15200,...,0.05470,0.1980,-0.0139,-0.8080,-0.324,0.16300,-0.14500,0.855,0.1160,-0.000114
NX7.N,0.1110,0.2100,-0.751,-0.335,0.000865,-1.3700,-0.2730,0.1060,-0.717,0.00865,...,0.17600,0.3960,-0.3640,-1.6400,-1.480,0.45300,-0.16800,0.411,-0.1760,0.218000


In [10]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = en.get_proteomics()
protfilter = proteomics.index.str.endswith('.N') #filter out normal samples
proteomics = proteomics[~protfilter]


prot_list = list(proteomics.columns)

mut_type = en.get_genotype_all_vars(gene)
mut_type = mut_type.drop(columns = ["Mutation_Status", "Location"])

prot_and_mutations = mut_type.join(proteomics)


# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = prot_and_mutations['Mutation'].isin(compare)
missense_wt = prot_and_mutations[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


endo_d = {}

for prot in proteomics:
    dif_endo = wt_med[prot] - missense_med[prot]
    endo_d[prot] = dif_endo
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(endo_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
endo_df = median_diff.join(t_test)
endo_df

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


,Difference_In_Median,P_Value
A1BG,-0.07100,0.720461
A2M,-0.10300,0.681688
A2ML1,-0.47135,0.136212
A4GALT,-0.22545,0.492356
AAAS,-0.07410,0.295868
...,...,...
ZXDC,-0.08700,0.808610
ZYG11B,0.03240,0.755896
ZYX,0.13800,0.251409
ZZEF1,0.00210,0.398421


# Brca get difference in median and t-test

In [11]:
br = cptac.Brca()

In [12]:
#separate proteomics into genes with duplicate database ID and genes with only one databaseID
proteomics = br.get_proteomics()
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
no_duplicates = proteomics[proteomics.columns[~fil]]

In [13]:
#FOR NO DUPLICATES
#get the pval and min for no duplicates
no_duplicates = br.reduce_multiindex(no_duplicates, flatten=True)

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
mut_type = br.get_genotype_all_vars(gene)
mut_type = mut_type.drop(columns = ["Mutation_Status", "Location"])

prot_and_mutations = mut_type.join(no_duplicates)


# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = prot_and_mutations['Mutation'].isin(compare)
missense_wt = prot_and_mutations[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


no_dup_d = {}

for prot in no_duplicates:
    dif_brca = wt_med[prot] - missense_med[prot]
    no_dup_d[prot] = dif_brca
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(no_dup_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
no_dup_df = median_diff.join(t_test)
no_dup_df

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)


,Difference_In_Median,P_Value
A1BG_NP_570602.2,-0.64290,0.605521
A2M_NP_000005.2,0.26040,0.917772
A2ML1_NP_653271.2|NP_001269353.1,0.74900,0.070253
AAAS_NP_056480.1|NP_001166937.1,-0.16450,0.093375
AACS_NP_076417.2|NP_001306769.1|NP_001306768.1,0.07180,0.899917
...,...,...
ZXDC_NP_079388.3|NP_001035743.1,-0.08945,0.583564
ZYG11B_NP_078922.1,0.13290,0.124112
ZYX_NP_001010972.1,-0.47940,0.426235
ZZEF1_NP_055928.3,0.02700,0.537838


In [14]:
#DEALING WITH DUPLICATES
#get the pval and min for duplicates

duplicates = br.reduce_multiindex(duplicates, flatten=True)

# We need to figure which of databaseID we want to use, for each protein
proteomics = br.get_proteomics()
protfilter = proteomics.index.str.endswith('.N') #filter out normal samples
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
duplicate_gene_names = list(set(duplicates.columns.get_level_values('Name'))) #get a list of genes names that are duplicated
duplicates = br.reduce_multiindex(duplicates, flatten=True)

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest

mut_type = br.get_genotype_all_vars(gene)
mut_type = mut_type.drop(columns = ["Mutation_Status", "Location"])

prot_and_mutations = mut_type.join(duplicates)


# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = prot_and_mutations['Mutation'].isin(compare)
missense_wt = prot_and_mutations[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


duplicates_d = {}

for prot in duplicates:
    dif_brca = wt_med[prot] - missense_med[prot]
    duplicates_d[prot] = dif_brca
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(duplicates_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
dup_df = median_diff.join(t_test)
dup_df

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)


,Difference_In_Median,P_Value
ABCB7_NP_001258628.1|NP_001258626.1,0.28370,0.253549
ABCB7_NP_004290.2|NP_001258625.1|NP_001258627.1,0.18180,0.641102
ABLIM3_NP_001287947.1|NP_001287944.1,-0.33190,0.852467
ABLIM3_NP_001287956.1,-0.37175,0.135988
ACIN1_NP_001158286.1,0.10270,0.800310
...,...,...
YBX3_NP_003642.3,-0.03920,0.559371
ZAK_NP_057737.2,-0.09030,0.780759
ZAK_NP_598407.1,-0.21270,0.989303
ZNF185_NP_001171579.1|NP_001171584.1,-0.19150,0.171569


In [15]:
#for each duplicated gene name select the one with the min p val and append to no_duplicat df
for name in duplicate_gene_names:
    gene_duplicate = dup_df[dup_df.index.str.contains(name)]
    #select the one with the min p value for t test
    min_row = gene_duplicate[gene_duplicate.P_Value == gene_duplicate.P_Value.min()]
    #append the selected version onto no_duplicate dataframe
    no_dup_df=no_dup_df.append(min_row)
no_dup_df=no_dup_df.sort_index()
no_dup_df

,Difference_In_Median,P_Value
A1BG_NP_570602.2,-0.64290,0.605521
A2ML1_NP_653271.2|NP_001269353.1,0.74900,0.070253
A2M_NP_000005.2,0.26040,0.917772
AAAS_NP_056480.1|NP_001166937.1,-0.16450,0.093375
AACS_NP_076417.2|NP_001306769.1|NP_001306768.1,0.07180,0.899917
...,...,...
ZXDC_NP_079388.3|NP_001035743.1,-0.08945,0.583564
ZYG11B_NP_078922.1,0.13290,0.124112
ZYX_NP_001010972.1,-0.47940,0.426235
ZZEF1_NP_055928.3,0.02700,0.537838


In [16]:
#reformat the table by getting rid of databaseIDs so we can join it to Endo and BRca
text='A2ML1_NP_653271.2|NP_001269353.1'
sep = '_'
text.split(sep)[0]

'A2ML1'

In [17]:
split = no_dup_df.index.str.split(sep, 1).str[0]
no_dup_df['new_index']=split
brca_df = no_dup_df.set_index("new_index")
# split["A"].str[0]

In [18]:
brca_df 

,Difference_In_Median,P_Value
new_index,,
A1BG,-0.64290,0.605521
A2ML1,0.74900,0.070253
A2M,0.26040,0.917772
AAAS,-0.16450,0.093375
AACS,0.07180,0.899917
...,...,...
ZXDC,-0.08945,0.583564
ZYG11B,0.13290,0.124112
ZYX,-0.47940,0.426235


# Colon get difference in median and t-test

In [19]:
cl = cptac.Colon()

In [20]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = cl.get_proteomics()
protfilter = proteomics.index.str.endswith('.N')
proteomics = proteomics[~protfilter]

prot_list = list(proteomics.columns)

mut_type = cl.get_genotype_all_vars(gene)
mut_type = mut_type.drop(columns = ["Mutation_Status", "Location"])

prot_and_mutations = mut_type.join(proteomics)


# Keep two values to compare
compare = ['Wildtype_Tumor','nonsynonymous SNV']
get = prot_and_mutations['Mutation'].isin(compare)
missense_wt = prot_and_mutations[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "nonsynonymous SNV"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


colon_d = {}

for prot in proteomics:
    dif_colon = wt_med[prot] - missense_med[prot]
    colon_d[prot] = dif_colon
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(colon_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
colon_df = median_diff.join(t_test)
colon_df

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  rejec

,Difference_In_Median,P_Value
A1BG,0.15300,0.873969
A1CF,0.33610,0.021796
A2M,0.07100,0.971735
AAAS,-0.04400,0.557351
AACS,-0.03700,0.452855
...,...,...
ZW10,0.00230,0.756621
ZWILCH,0.18430,0.233675
ZWINT,-0.32775,0.546515
ZYX,-0.13700,0.921572


# Combine Endo, Brca, and Colon into one table

In [21]:
brca_df=brca_df.rename_axis(None)

In [22]:
brca_endo = brca_df.join(endo_df, lsuffix='_Brca', rsuffix='_Endo')
brca_endo

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo
A1BG,-0.64290,0.605521,-0.07100,0.720461
A2M,0.26040,0.917772,-0.10300,0.681688
A2ML1,0.74900,0.070253,-0.47135,0.136212
AAAS,-0.16450,0.093375,-0.07410,0.295868
AACS,0.07180,0.899917,0.03900,0.734326
...,...,...,...,...
ZXDC,-0.08945,0.583564,-0.08700,0.808610
ZYG11B,0.13290,0.124112,0.03240,0.755896
ZYX,-0.47940,0.426235,0.13800,0.251409
ZZEF1,0.02700,0.537838,0.00210,0.398421


In [23]:
final_table = brca_endo.join(colon_df, rsuffix='_colon')
final_table

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
A1BG,-0.64290,0.605521,-0.07100,0.720461,0.153,0.873969
A2M,0.26040,0.917772,-0.10300,0.681688,0.071,0.971735
A2ML1,0.74900,0.070253,-0.47135,0.136212,NaN,NaN
AAAS,-0.16450,0.093375,-0.07410,0.295868,-0.044,0.557351
AACS,0.07180,0.899917,0.03900,0.734326,-0.037,0.452855
...,...,...,...,...,...,...
ZXDC,-0.08945,0.583564,-0.08700,0.808610,NaN,NaN
ZYG11B,0.13290,0.124112,0.03240,0.755896,NaN,NaN
ZYX,-0.47940,0.426235,0.13800,0.251409,-0.137,0.921572
ZZEF1,0.02700,0.537838,0.00210,0.398421,-0.016,0.391801


In [29]:
# final_table.to_csv("proteomics.csv")

In [24]:
# endo_and_colon = pd.concat([brca_df, endo_and_colon], axis=1)

In [26]:
# final_table = pd.concat([endo_and_colon, brca_df], axis=1, sort=False)

In [27]:
# final_table